# Registros de empleados

Se creara una aplicacion grafica con Python que gestione una base de datos de "empleados", entendiendose por gestion la Conexión a la base de datos, Creación de la tabla de empleados,Inserción, Consulta, actualización y Eliminación de un empleado y su informacion dentro de la empresa.

In [1]:
### Importacion de librerias
import sqlite3
from sqlite3 import Error
from tkinter import *
from tkinter import messagebox

# Creación de la conexión a la base de datos

In [2]:
def crear_conexion():
    conn = None;
    try:
        conn = sqlite3.connect('empleados_db.sqlite')
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn

# Creación de la tabla de empleados

In [3]:
def crear_tabla(conn):
    try:
        sql = ''' CREATE TABLE IF NOT EXISTS empleados (
                                        id integer PRIMARY KEY,
                                        nombre text NOT NULL,
                                        cargo text NOT NULL,
                                        salario real
                                    ); '''
        conn.cursor().execute(sql)
    except Error as e:
        print(e)

## Insercion de empleados

In [4]:
def insertar_empleado(conn, empleado):
    sql = ''' INSERT INTO empleados(nombre,cargo,salario)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, empleado)
    conn.commit()
    return cur.lastrowid

# Consulta de empleados

In [5]:
def consultar_empleados(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM empleados")
    rows = cur.fetchall()
    for row in rows:
        print(row)

# Actualización de empleados

In [6]:
# Actualización de empleados
def actualizar_empleado(conn, empleado):
    sql = '''
    UPDATE empleados
    SET nombre = ?,
        cargo = ?,
        salario = ?
    WHERE id = ?
    '''
    cur = conn.cursor()
    try:
        cur.execute(sql, empleado)
        conn.commit()
        return "Empleado actualizado correctamente."
    except Exception as e:
        print(f"Error al actualizar empleado: {e}")
        return f"Error al actualizar empleado: {e}"


# Eliminación de un empleado

In [7]:
# Eliminación de un empleado
def eliminar_empleado(conn, id):
    sql = 'DELETE FROM empleados WHERE id = ?'
    cur = conn.cursor()
    try:
        cur.execute(sql, (id,))
        conn.commit()
        return "Empleado eliminado correctamente."
    except Exception as e:
        print(f"Error al eliminar empleado: {e}")
        return f"Error al eliminar empleado: {e}"

# Creación de la ventana de la aplicación

In [8]:
def crear_ventana():
    ventana = Tk()
    ventana.title("Gestión de empleados")
    ventana.geometry("800x600")
    return ventana

# Creación de botones y campos de texto para la interfaz de usuario

In [9]:
from tkinter import messagebox

def crear_interfaz(ventana, conn):
    # Crear campos de texto para introducir información de empleados
    Label(ventana, text="Nombre del empleado:").pack()
    nombre_texto = StringVar()
    nombre_entry = Entry(ventana, textvariable=nombre_texto)
    nombre_entry.pack()

    Label(ventana, text="Cargo del empleado:").pack()
    cargo_texto = StringVar()
    cargo_entry = Entry(ventana, textvariable=cargo_texto)
    cargo_entry.pack()

    Label(ventana, text="Salario del empleado:").pack()
    salario_texto = StringVar()
    salario_entry = Entry(ventana, textvariable=salario_texto)
    salario_entry.pack()

    # Botón para insertar empleados
    boton_insertar = Button(ventana, text="Insertar empleado", 
                            command=lambda: insertar_empleado(conn, (nombre_texto.get(), cargo_texto.get(), float(salario_texto.get()))))
    boton_insertar.pack()

    # Crear un widget Text para mostrar los resultados de la consulta
    resultados_texto = Text(ventana)
    resultados_texto.pack()

    # Botón para consultar empleados
    def mostrar_empleados():
        # Limpiar el widget Text antes de mostrar nuevos resultados
        resultados_texto.delete(1.0, END)

        # Consultar empleados y mostrar los resultados
        cur = conn.cursor()
        cur.execute("SELECT * FROM empleados")
        rows = cur.fetchall()
        for row in rows:
            resultados_texto.insert(END, str(row) + '\n')

    boton_consultar = Button(ventana, text="Consultar empleados", command=mostrar_empleados)
    boton_consultar.pack()

    Label(ventana, text="ID del empleado a actualizar/eliminar:").pack()
    id_texto = StringVar()
    id_entry = Entry(ventana, textvariable=id_texto)
    id_entry.pack()

    # Botón para cargar empleados
    def cargar_empleado():
        cur = conn.cursor()
        cur.execute("SELECT * FROM empleados WHERE id=?", (int(id_texto.get()),))
        row = cur.fetchone()
        if row is not None:
            nombre_texto.set(row[1])
            cargo_texto.set(row[2])
            salario_texto.set(str(row[3]))
        else:
            mensaje.set("No se encontró un empleado con ese ID")

    boton_cargar = Button(ventana, text="Cargar empleado", command=cargar_empleado)
    boton_cargar.pack()

    # Crear un widget Label para mostrar mensajes
    mensaje = StringVar()
    mensaje_label = Label(ventana, textvariable=mensaje)
    mensaje_label.pack()

     # Botón para actualizar empleados
    def actualizar():
        respuesta = messagebox.askokcancel("Confirmar actualización", "¿Estás seguro de que quieres actualizar este empleado?")
        if respuesta:
            mensaje_actualizacion = actualizar_empleado(conn, (nombre_texto.get(), cargo_texto.get(), float(salario_texto.get()), int(id_texto.get())))
            mensaje.set(mensaje_actualizacion)

    boton_actualizar = Button(ventana, text="Actualizar empleado", command=actualizar)
    boton_actualizar.pack()

     # Botón para eliminar empleados
    def eliminar():
        respuesta = messagebox.askokcancel("Confirmar eliminación", "¿Estás seguro de que quieres eliminar este empleado?")
        if respuesta:
            mensaje_eliminacion = eliminar_empleado(conn, int(id_texto.get()))
            mensaje.set(mensaje_eliminacion)

    boton_eliminar = Button(ventana, text="Eliminar empleado", command=eliminar)
    boton_eliminar.pack()

if __name__ == '__main__':
    conn = crear_conexion()
    crear_tabla(conn)
    ventana = crear_ventana() 
    crear_interfaz(ventana, conn)
    ventana.mainloop()


2.6.0
